In [37]:
from mne.io import read_raw_edf
import mne
import pandas as pd
import numpy as np
from mne.preprocessing.ica import ICA

In [2]:
def find_channel(ch, chs_to_find):
    for ch_to_find in chs_to_find:
        if ch_to_find.lower() in ch.lower():
            return ch_to_find
    return ch

def add_montage(raw, ch_map=None):
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
    if ch_map:
        ten_twenty_montage.ch_names = [ch_map[ch] if ch in ch_map else ch
                                       for ch in ten_twenty_montage.ch_names]
    raw.set_montage(ten_twenty_montage)
    return raw

In [3]:
sample = read_raw_edf('test2.edf')

Extracting EDF parameters from C:\Users\Пользователь\Desktop\dev\test2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [4]:
chs_to_use = ['C3', 'C4', 'Cz', 'F3', 'F4', 'F7', 'F8', 'Fp1', 'Fp2', 'Fz',
               'O1', 'O2', 'P3', 'P4', 'Pz', 'T3', 'T4', 'T5', 'T6']

In [5]:
ch_map = {ch: find_channel(ch, chs_to_find=chs_to_use) for ch in sample.ch_names}

In [6]:
sample.rename_channels(ch_map);

In [7]:
sample.pick_channels(chs_to_use);

In [8]:
sample = add_montage(sample, ch_map=None)

In [9]:
mont = sample.get_montage()

### Сохраняем семпл

In [10]:
pd.DataFrame(sample.get_data()).to_csv('test2.csv', index=False, header=False)

In [11]:
with open('test2.xyz', 'w') as f:
    for i, (key, loc) in enumerate(mont.get_positions()['ch_pos'].items()):
        f.write(f'{i}\t{loc[0]}\t{loc[1]}\t{loc[2]}\t{key}\n')

### Сохраняем ICA

In [18]:
ica = ICA(n_components=15)

<ipython-input-18-2292f52926df>:1: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = ICA(n_components=15)


In [20]:
ica.fit(sample)

Fitting ICA to data using 19 channels (please be patient, this may take a while)
Selecting by number: 15 components


<ipython-input-20-2570303770f7>:1: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(sample)


Fitting ICA took 0.4s.


<ICA | raw data decomposition, fit (fastica): 28500 samples, 15 components, channels used: "eeg">

In [57]:
np.savetxt('test2_ica.csv', ica.get_components().T, delimiter='\t', fmt='%.12f')

### Читаем результаты из matlab

In [80]:
f'{1:02d}'

'01'

In [83]:
iclabel_preds = pd.read_csv('test2_iclabel.csv', header=None, 
                  names=['flag_brain', 'flag_muscles', 'flag_eyes', 'flag_heart', 'flag_line_noise', 'flag_ch_noise', 'flat_other'])
iclabel_preds.index = [f'ICA{i:03d}' for i in range(len(iclabel_preds))]

In [84]:
iclabel_preds

,flag_brain,flag_muscles,flag_eyes,flag_heart,flag_line_noise,flag_ch_noise,flat_other
ICA000,0.080687,0.001852,0.001511,0.001789,0.010031,0.000848,0.903281
ICA001,0.651608,0.002311,0.000742,0.000482,0.008329,0.000615,0.335913
ICA002,0.163483,0.000216,0.000271,0.000673,0.010732,0.019210,0.805415
ICA003,0.297324,0.034956,0.005871,0.012267,0.004329,0.022445,0.622809
ICA004,0.185386,0.030835,0.002514,0.002815,0.068894,0.006181,0.703375
ICA005,0.994844,0.000007,0.000019,0.000047,0.002235,0.000234,0.002614
ICA006,0.019974,0.002659,0.001005,0.001763,0.001836,0.004835,0.967927
ICA007,0.057851,0.008781,0.014888,0.002617,0.023688,0.006090,0.886085
ICA008,0.002919,0.000721,0.000095,0.000057,0.001964,0.000259,0.993985
ICA009,0.032303,0.002900,0.003842,0.001680,0.028995,0.007254,0.923026
